In [ ]:
import streamlit as st
import numpy as np
import pandas as pd
import joblib
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from PIL import Image

# Load the trained models
log_reg_model = joblib.load('C:/Users/potab/OneDrive - Solent University/COM 726 Dissertation/Logistic_Regression.joblib')
rf_model = joblib.load('C:/Users/potab/OneDrive - Solent University/COM 726 Dissertation/Random_Forest.joblib')
gb_model = joblib.load('C:/Users/potab/OneDrive - Solent University/COM 726 Dissertation/Gradient_Boosting.joblib')
nn_model = tf.keras.models.load_model('C:/Users/potab/OneDrive - Solent University/COM 726 Dissertation/Neural_Network.h5')

# Load the dataset to fit the scaler
file_path = 'C:/Users/potab/OneDrive - Solent University/COM 726 Dissertation/heart_failure_clinical_records_dataset.csv'
data = pd.read_csv(file_path)
X = data.drop(columns=['DEATH_EVENT'])

# Define top features for each model
top_features_log_reg = ['age', 'serum_creatinine', 'ejection_fraction', 'time']
top_features_rf = ['age', 'serum_creatinine', 'ejection_fraction', 'time']
top_features_gb = ['age', 'serum_creatinine', 'ejection_fraction', 'time']
top_features_nn = ['age', 'serum_creatinine', 'ejection_fraction', 'time']

# Load and display the welcoming image
image_url = "https://cdn.images.express.co.uk/img/dynamic/11/750x445/788234.jpg"
st.image(image_url, caption='Heart Failure Prediction', use_column_width=True)

# Add a title and description
st.title("Heart Failure Readmission Prediction App")
st.write("""
This application helps predict the likelihood of heart failure readmission based on clinical data. 
Select your input parameters and choose a model to see the prediction.
""")

# Sidebar for user input
st.sidebar.header('User Input Parameters')
def user_input_features():
    age = st.sidebar.number_input('Age', min_value=0, max_value=120, value=60)
    anaemia = st.sidebar.selectbox('Anaemia (0 or 1)', (0, 1))
    creatinine_phosphokinase = st.sidebar.number_input('Creatinine Phosphokinase', min_value=0, value=582)
    diabetes = st.sidebar.selectbox('Diabetes (0 or 1)', (0, 1))
    ejection_fraction = st.sidebar.number_input('Ejection Fraction', min_value=0, max_value=100, value=38)
    high_blood_pressure = st.sidebar.selectbox('High Blood Pressure (0 or 1)', (0, 1))
    platelets = st.sidebar.number_input('Platelets', min_value=0, value=265000)
    serum_creatinine = st.sidebar.number_input('Serum Creatinine', min_value=0.0, value=1.9)
    serum_sodium = st.sidebar.number_input('Serum Sodium', min_value=0, value=130)
    sex = st.sidebar.selectbox('Sex (0 or 1)', (0, 1))
    smoking = st.sidebar.selectbox('Smoking (0 or 1)', (0, 1))
    time = st.sidebar.number_input('Time', min_value=0, value=4)
    
    data = {
        'age': age,
        'anaemia': anaemia,
        'creatinine_phosphokinase': creatinine_phosphokinase,
        'diabetes': diabetes,
        'ejection_fraction': ejection_fraction,
        'high_blood_pressure': high_blood_pressure,
        'platelets': platelets,
        'serum_creatinine': serum_creatinine,
        'serum_sodium': serum_sodium,
        'sex': sex,
        'smoking': smoking,
        'time': time
    }
    features = pd.DataFrame(data, index=[0])
    return features

input_df = user_input_features()

# Display user input
st.subheader('User Input Parameters')
st.write(input_df)

# Model selection
model_choice = st.sidebar.selectbox('Select Model', ('Logistic Regression', 'Random Forest', 'Gradient Boosting', 'Neural Network'))

# Select appropriate features based on the chosen model
if model_choice == 'Logistic Regression':
    selected_features = top_features_log_reg
elif model_choice == 'Random Forest':
    selected_features = top_features_rf
elif model_choice == 'Gradient Boosting':
    selected_features = top_features_gb
elif model_choice == 'Neural Network':
    selected_features = top_features_nn

input_df_selected = input_df[selected_features]

# Scale the input data
scaler = StandardScaler()
scaler.fit(X[selected_features])
input_data_scaled = scaler.transform(input_df_selected)

# Prediction
if model_choice == 'Logistic Regression':
    pred = log_reg_model.predict(input_data_scaled)
elif model_choice == 'Random Forest':
    pred = rf_model.predict(input_data_scaled)
elif model_choice == 'Gradient Boosting':
    pred = gb_model.predict(input_data_scaled)
elif model_choice == 'Neural Network':
    pred = (nn_model.predict(input_data_scaled) > 0.5).astype("int32")

result = int(pred[0]) if model_choice != 'Neural Network' else int(pred[0][0])

# Display prediction result
st.subheader('Prediction')
st.write('Readmission: Yes' if result == 1 else 'Readmission: No')

# Customize the layout and theme (optional)
st.markdown(
    """
    <style>
    .reportview-container {
        background: #f0f0f5;
        color: #262730;
    }
    .sidebar .sidebar-content {
        background: #262730;
        color: white;
    }
    </style>
    """,
    unsafe_allow_html=True
)

# Display footer message
st.sidebar.markdown("Developed by [Yetunde Adegunley](https://yourportfolio.com)")
